In [112]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rc

from lowAltitude_classification.gsd_utils import papermode

In [113]:
font_size = 8
papermode(plt, font_size, has_latex=True)
rc('text.latex', preamble=r'\usepackage{color}')
matplotlib.use('ps')

In [298]:
# data = pd.read_csv('results/scaling-test-METRICS.csv')
# data = pd.read_csv('results/scaling-test-METRICS_version2.csv')
data = pd.read_csv('results/scaling-test-METRICS_versionFinal.csv')

num_images = 143065
data['F1'] = data['F1'] * 100

factors = []
fractions = []
for idx, row in data.iterrows():
    fraction = str(row['factor']).replace('.', '/')
    factor = eval(fraction)
    factors.append(factor)
    fractions.append(fraction)
data['factor'] = factors
data['fraction'] = fractions

data['num_images'] = (num_images * data['factor']).astype(int)
data = data.sort_values('num_images')

data

,factor,F1,pAcc,fraction,num_images
10,0.001953,20.95,0.3403,1/512,279
9,0.003906,27.97,0.4052,1/256,558
8,0.007812,32.23,0.4428,1/128,1117
7,0.015625,34.89,0.4703,1/64,2235
6,0.031250,35.12,0.4791,1/32,4470
5,0.062500,35.75,0.4606,1/16,8941
4,0.125000,38.04,0.4728,1/8,17883
3,0.250000,37.69,0.4891,1/4,35766
2,0.500000,37.82,0.4860,1/2,71532
1,0.750000,38.93,0.4936,3/4,107298


In [299]:


width = 3.281
height = width / 1.618
height = 1.8
fig, ax = plt.subplots(figsize=(width, height))

ax.plot(data['num_images'], data['F1'], color='teal', marker='o')
ax.set_xlabel('Training Set Size (\# images)')
# ax.set_xlabel('Fraction of dataset')
ax.set_ylabel(r"$F1$ Score on \hspace{2.2em} (\%)")
ax.text(-0.113, 0.715, r"$D_{test}^{drone}$", fontsize=font_size, color="blue", transform=ax.transAxes, ha='center',
        va='center', rotation=90)
ax.set_xscale('log', base=2)
# ax.set_yscale('log')

labels = []
for i, (n, f) in enumerate(zip(data['num_images'], data['fraction'])):
    label = rf'{n}'"\n"rf'$\left({f}\right)$'
    if f == '1/1':
        label = rf'{n}'"\n"'$(1)$'
    if i % 2 == 0:
        labels.append(label)
    elif i == 7:
        labels.append(f'\n\n\n{label}')
    else:
        labels.append(f'\n\n{label}')
    # if f == '1/1':
    #     labels.append(rf'{n}'"\n"'$(1)$')
    # else:
    #     labels.append(rf'{n}'"\n"rf'$\left({f}\right)$')
# ax.set_yticks(range(34, 46, 2))
ax.set_xticks(data['num_images'], labels)  #, rotation=45, ha='right')
ax.grid(True, which="major", ls="--", color='gray')

ax.hlines(40.69, 200, 1e6, linestyles='dashed', color='orange', label='Pseudo-labels')
ax.hlines(26.83, 200, 1e6, linestyles='dashed', color='red', label='Supervised')

# Text top of the lines
ax.text(1400, 35, 'Pseudo-labels', color='orange', ha='right', backgroundcolor='white', bbox=dict(facecolor='white', alpha=1, edgecolor='none', boxstyle='round,pad=0.'))
ax.text(160000, 22, 'Supervised', color='red', ha='right', backgroundcolor='white', bbox=dict(facecolor='white', alpha=1, edgecolor='none', boxstyle='round,pad=0.'))

ax.set_xlim(200, num_images + 50000)
ax.set_ylim(15, 50)
# fig.legend(loc='lower left')

fig.subplots_adjust(top=0.92, bottom=0.45, left=0.12, right=0.99)
# fig.set_size_inches(width, 0.8 * height)

fig.savefig('results/M2F_scaling.pdf')
fig.savefig('results/M2F_scaling.png')

fig.show()

/tmp/ipykernel_7445/671667813.py:51: UserWarning: FigureCanvasPS is non-interactive, and thus cannot be shown
  fig.show()
